# Modeling
This notebook shows how I trained the model I use in my painting style recognition [app](http://www.nicolascontreras.tech/portfolio/painting_reco).

# 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import despine

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from glob import glob
from tqdm import tqdm
import os
import pickle

import tensorflow as tf

# 2. ImageDataGenerator

In [ ]:
def get_ImageDataGenerator():
    return tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
        rotation_range=30, brightness_range=(.75, 1.25), horizontal_flip=True
    )

def get_generator(image_data_generator, directory, seed=None):
    return image_data_generator.flow_from_directory(directory=directory,
                                                    batch_size=20,
                                                    target_size=(299, 299),
                                                    class_mode='categorical',
                                                    seed=seed)

In [ ]:
# Instanciate generator
img_gen = get_ImageDataGenerator()

# Set dir paths for train & valid data
train_path = os.path.join('data', 'train')
valid_path = os.path.join('data', 'valid')

# Flow generators from dir paths
train_generator = get_generator(img_gen, train_path)
valid_generator = get_generator(img_gen, valid_path)

# 3. Build model

In [ ]:
inception_v3 = tf.keras.applications.InceptionV3(weights='imagenet')

feature_extractor = tf.keras.models.Model(
    inputs = inception_v3.input,
    outputs = inception_v3.layers[-1].input
)

feature_extractor.trainable = False

inputs = inception_v3.input
h = feature_extractor(inputs)

h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Dense(128, activation='relu')(h)

h = tf.keras.layers.BatchNormalization()(h)
outputs = tf.keras.layers.Dense(70, activation='softmax')(h)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

# 4. Train model

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=.5)
]
with tf.device('/gpu:0'):
    history = model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=callbacks)

# 5. Evaluate model performance

In [ ]:
def style_plot(ax, legend=True):
    plt.grid(axis='y', **{'color': 'tab:gray', 'linewidth': 1, 'alpha': .25})
    plt.tick_params(left=False, bottom=False, axis='y', colors='tab:gray')
    plt.xlabel('Epochs', fontsize=12)
    plt.yticks(fontsize=12)
    if legend:
        plt.legend(frameon=True, framealpha=1, facecolor='white', edgecolor='white', fontsize=12)
    ax.spines['bottom'].set_color('tab:gray')
    
plt.figure(figsize=(15, 8))

ax = plt.subplot(211)
plt.plot(history.history['lr'],  lw=2)
plt.title('Learning rate', fontsize=16)
style_plot(ax, False)

ax = plt.subplot(223)
plt.plot(history.history['accuracy'], lw=2, label='Training accuracy')
plt.plot(history.history['val_accuracy'], lw=2, label='Validation accuracy')
plt.title('Accuracies', fontsize=16)
style_plot(ax)

ax = plt.subplot(224)
plt.plot(history.history['loss'], lw=2, label='Training loss')
plt.plot(history.history['val_loss'], lw=2, label='Validation loss')
plt.title('Losses', fontsize=16)
style_plot(ax)

despine(left=True)
plt.tight_layout(w_pad=10, h_pad=5)
plt.show()

# 6. Save model and model history on disk

In [ ]:
model.save('painting_style_reco.h5')

In [ ]:
filepath = os.path.join('model_history.pkl')
with open(filepath, 'wb') as f:
    pickle.dump(models_history, f)